# Cutout data from fits files

## Imports

In [ ]:
import sys
import os
import pprint
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.multilens import MultiLens
from gleam.utils.plotting import IPColorbar, IPPointCache
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()

## Main

### Search for a fits file

In [ ]:
fidx = 5
files = !ls data/*/*[0-9]_field.fits
pprint.pprint(files)
filename = files[fidx]
lensname = os.path.basename(filename).split('.')[0]
print("Selected file: {}".format(lensname))

### Read fits file and extract data

In [ ]:
hdu = fits.open(filename)
for h in hdu:
    if isinstance(h, fits.hdu.image.ImageHDU):
        hdu = h
        break
if isinstance(hdu, list):
    hdu = hdu[0]
print(hdu)
print(hdu.data.shape)
print(repr(hdu.header))

position = [p//2 for p in hdu.data.shape]

In [ ]:
lo = LensObject(filename, auto=False, glscfactory_options={})
print(lo.__v__)

### Plot the data

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots()
fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.vilux, colorbar=True,
                             vmin=0, vmax=0.5*np.max(lo.data), source_images=False,
                             deconv=False, psf=np.ones((3, 3))/25.)
clrbar = IPColorbar(plt_out[1], plt_out[0])
clrbar.connect()
cache = IPPointCache(plt_out[0])
cid = cache.connect()

In [ ]:
if cache.xy:
    position = cache.xy[-1]
else:
    position = None
plt.close()
print("Cutout center @ {}".format(position))

In [ ]:
fs = fig.axes[0].viewLim
delta = int(0.5 + max(abs(fs.x1-fs.x0), abs(fs.y1-fs.y0)))
delta = int(np.ceil(delta / 99.0)) * 99
size = (delta, delta) if delta < hdu.data.shape[0] else hdu.data.shape
if size == hdu.data.shape:
    position = tuple(s//2 for s in hdu.data.shape)
print("position: {}".format(position), "size: {}".format(size))

### Ovewrite fits file and plot again

In [ ]:
wcs = WCS(hdu.header)
cutout = Cutout2D(hdu.data, position, size, wcs)
hdu.data = cutout.data
hdu.header.update(cutout.wcs.to_header())

In [ ]:
# show again for validation
%matplotlib notebook

fig, ax = plt.subplots()
pltdta = hdu.data[:]
# pltdta = np.log10(pltdta+1+np.abs(np.nanmin(pltdta)))
plt_out = ax.imshow(pltdta, cmap=gcl.GLEAMcmaps.vilux, vmin=0, vmax=0.5*np.max(pltdta))
cbar = plt.colorbar(plt_out, cax=ax)
clrbar = IPColorbar(cbar, plt_out)
clrbar.connect()

### Save file

In [ ]:
savename = "{}.fits".format(os.path.dirname(filename.replace('_field', '')))
print("Saving {}...".format(savename))

In [ ]:
hdu.writeto(savename, overwrite=True)